**Проект по автоматизации**

В целях выполнения проекта по созданию дашборда необходимо: 
- подключиться к базе содержащей raw data
- провести проверку и анализ данных в базе
- выполнить выгрузку данных в csv файл 
Далее проект будет выполняться в tableau

Загружаем необходимые библиотеки и выполняем подключение:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

Загрузка данных:

In [2]:
query = ''' SELECT *
            FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con = engine)

pd.io.sql.read_sql(query, con = engine)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,1071337,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,1071338,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,1071339,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,1071340,Юмор,Финансы,41-45,2019-09-24 18:56:00,1


Просмотр датасета:

In [3]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


Датасет состоит из 6 колонок:
- колонка record_id содержащая id операции. Тип данных integer
- колонка item_topic информация с темами карточек. Тип данных object
- колонка source_topic информация с темами источника. Тип данных object
- колонка age_segment с информацией о возрастных группах. Тип данных также object
- колонка веб с информацией о дате и времени события. Тип данных datetime 
- колонка visits с информацией о кол-ве событий

Проверка на дубликаты:

In [4]:
dash_visits.duplicated().sum()

0

Почитаем кол-во уникальных id операций:

In [16]:
dash_visits['record_id'].nunique()

30745

In [6]:
dash_visits['record_id'].nunique()

30745

Просмотрим "строковые" колонки 'item_topic','source_topic','age_segment':

In [21]:
dash_visits['item_topic'].value_counts().sort_values(ascending=False)

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

In [23]:
dash_visits['source_topic'].value_counts().sort_values(ascending=False)

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

In [27]:
dash_visits['age_segment'].value_counts().sort_values(ascending=False)

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

Посмотрим за какой временной промежуток предоставлены данные:

In [25]:
dash_visits['dt'].min()

Timestamp('2019-09-24 18:28:00')

In [26]:
dash_visits['dt'].max()

Timestamp('2019-09-24 19:00:00')

Данные представлены за время с 18:28 до 19:00 24 сентября 2019

In [24]:
dash_visits['visits'].sum()

310207

Общая сумма операций 310207

**Экспорт данных в csv файл:**

In [14]:
dash_visits.to_csv('dash_visits.csv', index=False)

Проверка:

In [15]:
test_visits = pd.read_csv('dash_visits.csv')
test_visits

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,1071337,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,1071338,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,1071339,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,1071340,Юмор,Финансы,41-45,2019-09-24 18:56:00,1


Экспорт данных произведен успешно.

Дальнейшая работа будет проведена в Tableau.